# Mel-Frequency Cepstral Coefficients 

## Read Data

In [1]:
import numpy as np
signal = np.random.rand(70000)
sample_rate = 16000 #16kHz

## Framing data

In [2]:
# frame length : 20-40ms eg.25ms 
fram_stride = 0.01
fram_size = 0.025
fram_length, fram_step = fram_size * sample_rate, fram_stride * sample_rate
signal_length = len(signal)
fram_length = int(round(fram_length))
fram_step = int(round(fram_step))
num_frames = int(np.ceil(float(np.abs(signal_length - fram_length + fram_step)) / fram_step)) #make sure that we have at least 1 frame

In [3]:
#填充信号以确保所有帧具有相同数量的样本，且不会截断原始信号中的任何样本
pad_signal_lenth = (num_frames - 1) * fram_step + fram_length
z = np.zeros(pad_signal_lenth - signal_length)
pad_signal = np.append(signal,z)

In [4]:
print(fram_length)
print(num_frames)
print(len(pad_signal))

400
436
70000


## Quantizing data fram

In [5]:
indices = np.tile(np.arange(0,fram_length),(num_frames,1)) + np.tile(np.arange(0,num_frames * fram_step, fram_step),(fram_length,1)).T
print(indices.shape)
frames = pad_signal[indices.astype(np.int32, copy = False)]

(436, 400)


## Discre Fourier Transform

In [6]:
#S_i(k) = sum_(N,n=1)(s_i(n) * h(n) * exp(-j * 2pi / N * k * n)), 1<k<K; K-DFT length, N - Sample long, h(n) - analysis window
frames *= np.hamming(fram_length) #对每一帧加汉明窗窗口函数 w[n] = 0.54-0.46cos(2pi(n/(N-1))) 主要抵消FFT无限计算并减少谱泄露

In [7]:
NFFT = 512 #N点FFT 也就是 DFT N通常为256或512
mag_frames = np.absolute(np.fft.rfft(frames,NFFT)) # magnitude of FFT
print(mag_frames.shape)
pow_fram = ((1.0/NFFT) * ((mag_frames) ** 2)) # power spectrum of FFT

(436, 257)


## Mel-spaced filterbank

### Mel-scale calculation

In [8]:
nfilt = 26 #difine the number of triangular filters range between 20 and 40 ( common amount is 26)
low_freq_mel = 0
hight_freq_mel = (2595 * np.log10(1 + (sample_rate / 2) / 700)) # M(f) = 2595*lg(1 + f / 700) convert Hz to Mel
mel_point = np.linspace(low_freq_mel, hight_freq_mel, nfilt + 2)# equally spaced in mel scale, 划分出nfilt + 2 个区间
hz_points = (700 * (10 ** (mel_point / 2595) - 1)) # convert Mel to Hz
bin = np.floor((NFFT + 1) * hz_points / sample_rate)

### Filter design

In [12]:
fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
print(fbank.shape)
for m in range(1, nfilt + 1):
    f_m_minus = int(bin[m - 1])
    f_m = int(bin[m]) 
    f_m_plus = int(bin[m + 1])
    
    for k in range(f_m_minus,f_m):
        fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
    for k in range(f_m,f_m_plus):
        fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])

(26, 257)


## log magnitude calculation

In [13]:
filter_banks = np.dot(pow_fram, fbank.T)
filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)
filter_banks = 20 * np.log10(filter_banks) #dB give 26 log filterbank energies

## Discrete Cosine Transform calculation

In [18]:
from scipy.fftpack import dct
num_ceps = 12
mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, -(num_ceps + 1):] # the least 12-13 cepstral coefficients are used

cep_lifter = 22
(nframes, ncoeff) = mfcc.shape
n = np.arange(ncoeff)
lift = 1 + (cep_lifter / 2) * np.sin(np.pi * n/cep_lifter)
mfcc *= lift

mfcc.shape

(436, 13)